<a href="https://colab.research.google.com/github/Chethan-Kumar9845/Plastic-Waste-Detection/blob/main/Plastic_Waste_Detection_Using_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gdown

import gdown
import zipfile

file_id = "18pI0FdziEH-pm-eyf49dbqI062OYwIrJ"
download_url = f"https://drive.google.com/uc?id={file_id}"
output_zip = "Dataset.zip"

gdown.download(download_url, output_zip, quiet=False)

with zipfile.ZipFile(output_zip, "r") as zip_ref:
    zip_ref.extractall("plastic_dataset")

print("Dataset downloaded and extracted to 'plastic_dataset' folder.")


In [ ]:
import os

# List all items (including folders and files) in plastic_dataset
print("plastic_dataset contents:", os.listdir('plastic_dataset'))

# List all items in plastic_images (should show your files/folders)
print("plastic_images contents:", os.listdir('plastic_dataset/plastic_images'))


In [ ]:
image_folder = 'plastic_dataset/plastic_images/plastic_images'


In [ ]:
for root, dirs, files in os.walk('plastic_dataset'):
    for file in files:
        if file.lower().endswith(('.jpg','.jpeg','.png')):
            print(os.path.join(root, file))


In [ ]:
image_folder = 'plastic_dataset/plastic_images/plastic_images'
img_list = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg','.jpeg','.png'))]

print("Total images:", len(img_list))
print("First 5 images:", img_list[:5])


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

for i in range(5):
    img_path = os.path.join(image_folder, img_list[i])
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()



In [ ]:
import os

annotation_folder = 'plastic_dataset/plastic_annotation'
annotation_files = os.listdir(annotation_folder)
print("Total annotation files:", len(annotation_files))
print("First 5 annotation files:", annotation_files[:5])


In [ ]:
import os

annotation_folder = 'plastic_dataset/plastic_annotation/plastic_annotation'
annotation_files = os.listdir(annotation_folder)

print("Total annotation files:", len(annotation_files))
print("First 5 annotation files:", annotation_files[:5])

file_path = os.path.join(annotation_folder, annotation_files[0])
with open(file_path, 'r') as f:
    data = f.read()
print(data[:500])  # Print first 500 characters to inspect the structure


In [ ]:
import os
import xml.etree.ElementTree as ET

annotation_folder = 'plastic_dataset/plastic_annotation/plastic_annotation'
annotation_files = os.listdir(annotation_folder)

# Example: Parse first annotation file
file_path = os.path.join(annotation_folder, annotation_files[0])
tree = ET.parse(file_path)
root = tree.getroot()

# Get image filename
filename = root.find('filename').text

# Get object details
for obj in root.findall('object'):
    label = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = float(bndbox.find('xmin').text)
    ymin = float(bndbox.find('ymin').text)
    xmax = float(bndbox.find('xmax').text) if bndbox.find('xmax') is not None else None
    ymax = float(bndbox.find('ymax').text) if bndbox.find('ymax') is not None else None
    print(f"Filename: {filename}, Label: {label}, BoundingBox: ({xmin}, {ymin}), ({xmax}, {ymax})")


In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

annotation_folder = 'plastic_dataset/plastic_annotation/plastic_annotation'
annotation_files = os.listdir(annotation_folder)

data_list = []

for ann_file in annotation_files:
    file_path = os.path.join(annotation_folder, ann_file)
    tree = ET.parse(file_path)
    root = tree.getroot()
    filename = root.find('filename').text
    for obj in root.findall('object'):
        label = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text) if bndbox.find('xmax') is not None else None
        ymax = float(bndbox.find('ymax').text) if bndbox.find('ymax') is not None else None
        data_list.append([filename, label, xmin, ymin, xmax, ymax])

# Convert to DataFrame
df = pd.DataFrame(data_list, columns=['filename', 'label', 'xmin', 'ymin', 'xmax', 'ymax'])
print(df.head())


In [ ]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

image_folder = 'plastic_dataset/plastic_images/plastic_images'

for i in range(5):  # Display first 5 images with bounding boxes
    row = df.iloc[i]
    img_path = os.path.join(image_folder, row['filename'])
    img = Image.open(img_path)
    draw = ImageDraw.Draw(img)
    # Draw box
    draw.rectangle([row['xmin'], row['ymin'], row['xmax'], row['ymax']], outline='red', width=3)
    plt.imshow(img)
    plt.title(f"Label: {row['label']}")
    plt.axis('off')
    plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# Split based on unique filenames to avoid data leakage
unique_files = df['filename'].unique()
train_files, test_files = train_test_split(unique_files, test_size=0.2, random_state=42)

# Create train and test DataFrame subsets
train_df = df[df['filename'].isin(train_files)].reset_index(drop=True)
test_df = df[df['filename'].isin(test_files)].reset_index(drop=True)

print("Training samples:", len(train_df))
print("Testing samples:", len(test_df))


In [ ]:
import os
import shutil

os.makedirs('yolo_dataset/images/val', exist_ok=True)

# Move uploaded files to validation images folder
for filename in uploaded.keys():
    shutil.move(filename, f'yolo_dataset/images/val/{filename}')


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


In [ ]:
custom_data = """
train: ../yolo_dataset/images/train
val: ../yolo_dataset/images/val

nc: 1
names: ['plastic']
"""

with open('custom_data.yaml', 'w') as f:
    f.write(custom_data)


In [ ]:
import os
from PIL import Image

def convert_to_yolo(df, img_folder, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    grouped = df.groupby('filename')
    for filename, group in grouped:
        img_path = os.path.join(img_folder, filename)
        img = Image.open(img_path)
        img_width, img_height = img.size
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_path = os.path.join(save_folder, txt_filename)

        with open(txt_path, 'w') as f:
            for _, row in group.iterrows():
                x_center = ((row['xmin'] + row['xmax']) / 2) / img_width
                y_center = ((row['ymin'] + row['ymax']) / 2) / img_height
                width = (row['xmax'] - row['xmin']) / img_width
                height = (row['ymax'] - row['ymin']) / img_height
                f.write(f"0 {x_center} {y_center} {width} {height}\n")  # '0' is class index for plastic


In [ ]:
import os

# Paths for label folders where YOLO txt labels will be saved
label_train_folder = 'yolo_dataset/labels/train'
label_val_folder = 'yolo_dataset/labels/val'

# Paths for images copied into YOLOv5 format
image_train_folder = 'yolo_dataset/images/train'
image_val_folder = 'yolo_dataset/images/val'

# Create all necessary directories if they don't exist
os.makedirs(label_train_folder, exist_ok=True)
os.makedirs(label_val_folder, exist_ok=True)
os.makedirs(image_train_folder, exist_ok=True)
os.makedirs(image_val_folder, exist_ok=True)

print(f"Directories ensured:\n{label_train_folder}\n{label_val_folder}\n{image_train_folder}\n{image_val_folder}")


In [ ]:
import os

print("yolo_dataset contents:", os.listdir('yolo_dataset'))
# If there are subfolders
for root, dirs, files in os.walk('yolo_dataset'):
    print("Root:", root)
    for d in dirs:
        print(" - Dir:", d)
    for f in files[:5]:  # Show sample files
        print("   * File:", f)


In [ ]:
image_train_folder = 'yolo_dataset/images/train'
image_val_folder = 'yolo_dataset/images/val'
label_train_folder = 'yolo_dataset/labels/train'
label_val_folder = 'yolo_dataset/labels/val'


In [ ]:
import os

# Get all present image filenames in train and val folders
existing_train_images = set(os.listdir(image_train_folder))
existing_val_images = set(os.listdir(image_val_folder))

# Filter train/test df to only include images that exist in relevant folders
filtered_train_df = train_df[train_df['filename'].isin(existing_train_images)].reset_index(drop=True)
filtered_test_df = test_df[test_df['filename'].isin(existing_val_images)].reset_index(drop=True)

# Convert only for images that are present
convert_to_yolo(filtered_train_df, image_train_folder, label_train_folder)
convert_to_yolo(filtered_test_df, image_val_folder, label_val_folder)


In [ ]:
convert_to_yolo(filtered_train_df, image_train_folder, label_train_folder)
convert_to_yolo(filtered_test_df, image_val_folder, label_val_folder)


In [ ]:
import shutil
import os

original_image_folder = 'plastic_dataset/plastic_images/plastic_images'

# Copy training images based on train_df (unfiltered)
for img_file in train_df['filename'].unique():
    src = os.path.join(original_image_folder, img_file)
    dst = os.path.join('yolo_dataset/images/train', img_file)
    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"Missing train file: {src}")

# Copy validation images based on test_df (unfiltered)
for img_file in test_df['filename'].unique():
    src = os.path.join(original_image_folder, img_file)
    dst = os.path.join('yolo_dataset/images/val', img_file)
    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"Missing val file: {src}")


In [ ]:
print("Train images after copy:", os.listdir('yolo_dataset/images/train')[:5])
print("Val images after copy:", os.listdir('yolo_dataset/images/val')[:5])


In [ ]:
convert_to_yolo(train_df, original_image_folder, label_train_folder)
convert_to_yolo(test_df, original_image_folder, label_val_folder)


In [ ]:
print("Train images:", os.listdir('yolo_dataset/images/train'))
print("Val images:", os.listdir('yolo_dataset/images/val'))


In [ ]:
custom_data = """
train: yolo_dataset/images/train
val: yolo_dataset/images/val
nc: 1
names: ['plastic']
"""
with open('custom_data.yaml', 'w') as f:
    f.write(custom_data)


In [ ]:
!python train.py --img 640 --batch 16 --epochs 20 --data custom_data.yaml --weights yolov5s.pt --cache


In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.12 --source yolo_dataset/images/val --project runs/detect --name val_results --exist-ok


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

results_folder = 'runs/detect/val_results'
images = [f for f in os.listdir(results_folder) if f.endswith('.jpg')]

for image in images[:5]:
    img_path = os.path.join(results_folder, image)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(image)
    plt.show()


In [ ]:
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import torch

# Load your trained YOLOv5 model (adjust the path to your weights file)
model = torch.hub.load("ultralytics/yolov5", "custom", path="runs/train/exp3/weights/best.pt", force_reload=True)

# Create a file upload widget that accepts image files
upload_widget = widgets.FileUpload(accept='image/*', multiple=False)

def on_upload_change(change):
    clear_output(wait=True)
    for name, file_info in upload_widget.value.items():
        img_bytes = file_info['content']
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

        # Display uploaded image resized
        img_resized = img.resize((480, 480))
        display(img_resized)

        # Run inference
        results = model(img)

        # Get detection results as PIL image
        result_img = Image.fromarray(results.render()[0])

        # Resize detection output for display
        result_resized = result_img.resize((480, 480))
        display(result_resized)

        # If no detections, print message
        if len(results.xyxy[0]) == 0:
            print("No objects detected. Try uploading an image with plastic objects similar to the training data.")



# Set the function to trigger upon file upload
upload_widget.observe(on_upload_change, names='value')

# Display instructions and the upload widget
display(widgets.HTML("<h2>Upload an image to detect plastic waste</h2>"))
display(upload_widget)
